In [1]:
import pandas as pd
import json
import glob
from collections import Counter
from typing import List, Dict

In [2]:
pd.set_option('display.max_colwidth', None)

def df_to_latex(df: pd.DataFrame) -> None:
    print(df.to_latex(index=False))

In [3]:
def get_library_classes(library_name: str, library_dir: str, project_dir: str) -> pd.DataFrame:
    with open(library_dir, "r", encoding="utf-8") as library_file:
        library_data = json.load(library_file)

    # Get Most used Class
    classes = []

    for project in glob.glob(project_dir):
        with open(project, "r", encoding="utf-8") as project_file:
            project_data = json.load(project_file)

            for file in project_data.keys():
                file_data = project_data[file]
                for library in file_data.keys():
                    if library == library_name:
                        module_data = file_data[library]
                        for key, _ in module_data.items():
                            if key[0].isupper():
                                class_name_parts = key.split("_")
                                if len(class_name_parts) > 2:
                                    class_name = "_".join(class_name_parts[:-1])
                                else:
                                    class_name = class_name_parts[0]
                                classes.append(class_name)

    class_data = Counter(classes)
    df_classes = pd.DataFrame.from_dict(class_data, orient="index").reset_index()
    df_classes = df_classes.rename(columns={'index':'Class', 0:'Count'})
    df_classes = df_classes.sort_values(by=['Count'], ascending=False)

    # Get Number of API Options and categories
    class_options = []
    categories = []
    classes = df_classes["Class"].to_list()

    for ml_class in classes:
        try:
            class_data = next(filter(lambda x: x["name"] == ml_class, library_data))
            class_options.append(len(class_data["params"]))
            parts = class_data["full_name"].split(".")
            if len(parts) <= 2:
                category = parts[0]
            else:
                category = parts[1]
            categories.append(category)
        except StopIteration:
            print("Could not find: ", ml_class, library_name)
            class_options.append(0)
            categories.append("Unknown")
            continue

    df_classes["Category"] = categories
    df_classes["#HP"] = class_options

    # Compute average number of options used per class and most used option
    avg_class_options = []
    most_used_class_option = []

    for ml_class in classes:
        avg_class_options_used = []
        class_options = []
        for project in glob.glob(project_dir):
            with open(project, "r", encoding="utf-8") as project_file:
                project_data = json.load(project_file)

                for file in project_data.keys():
                    file_data = project_data[file]
                    for library in file_data.keys():
                        if library == library_name:
                            module_data = file_data[library]
                            for module_name, data in module_data.items():
                                if module_name[0].isupper():
                                    name_parts = module_name.split("_")
                                    if len(name_parts) > 2:
                                        name = "_".join(name_parts[:-1])
                                    else:
                                        name = name_parts[0]

                                    if ml_class == name:
                                        counter = 0
                                        for param in data.keys():
                                            if param == "variable":
                                                continue

                                            if  param not in ("variable", "params", "class"):
                                                counter += 1

                                            if param == "params":
                                                class_options.append("default")
                                            else:
                                                class_options.append(param)
            
                                        avg_class_options_used.append(counter)

                              
        avg_class_options.append(round((sum(avg_class_options_used) / len(avg_class_options_used)),2))
        class_option_data = Counter(class_options)
        try:
            most_common_number = class_option_data.most_common(1)[0][1]
            options = [elem[0] for elem in class_option_data.most_common() if elem[1] == most_common_number]
            option_str = ", ".join(options)
            most_used_class_option.append(option_str)
        except IndexError:
            most_used_class_option.append("None")

    df_classes["AvgOptionsUsed"] = avg_class_options
    df_classes["Most Used HP"] = most_used_class_option
    
    return df_classes

In [4]:
df_sklearn_classes = get_library_classes("sklearn", "../data/library_data/sklearn_data.json" , "../data/statistics/*")
df_sklearn_classes = df_sklearn_classes[:10]
df_to_latex(df=df_sklearn_classes)


\begin{tabular}{lrlrrl}
\toprule
             Class &  Count &        Category &  \#HP &  AvgOptionsUsed & Most Used HP \\
\midrule
    StandardScaler &    192 &   preprocessing &    3 &            0.12 &      default \\
               PCA &    136 &   decomposition &    9 &            1.23 & n\_components \\
            KMeans &    134 &         cluster &    9 &            2.28 &   n\_clusters \\
LogisticRegression &    124 &    linear\_model &   15 &            2.40 &            C \\
              TSNE &     98 &        manifold &   16 &            2.74 & n\_components \\
             KFold &     98 & model\_selection &    3 &            2.47 &     n\_splits \\
  LinearRegression &     85 &    linear\_model &    5 &            0.36 &      default \\
      LabelEncoder &     71 &   preprocessing &    0 &            0.00 &      default \\
      MinMaxScaler &     67 &   preprocessing &    3 &            0.42 &      default \\
               SVC &     65 &             svm &   15 &      

C:\Users\Basti\AppData\Local\Temp\ipykernel_20628\4188038665.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False))


In [5]:
df_tf_classes = get_library_classes("tensorflow", "../data/library_data/tensorflow_data.json" , "../data/statistics/*")
df_tf_classes = df_tf_classes[:10]
df_to_latex(df=df_tf_classes)

\begin{tabular}{lrlrrl}
\toprule
         Class &  Count &   Category &  \#HP &  AvgOptionsUsed &         Most Used HP \\
\midrule
      Variable &   2007 & tensorflow &   12 &            1.98 &        initial\_value \\
       Session &   1572 &     compat &    3 &            0.58 &              default \\
         Dense &   1554 &      keras &   11 &            2.72 &                units \\
         Saver &   1002 &     compat &   15 &            0.68 &              default \\
 AdamOptimizer &    909 &     compat &    6 &            1.41 &        learning\_rate \\
 DEFINE\_string &    836 &     compat &    6 &            3.00 &  name, default, help \\
   ConfigProto &    763 &     compat &   17 &            1.21 & allow\_soft\_placement \\
       Dropout &    693 &      keras &    4 &            1.03 &                 rate \\
DEFINE\_integer &    654 &     compat &    8 &            3.00 &  name, default, help \\
   TensorShape &    612 & tensorflow &    1 &            1.00 &        

C:\Users\Basti\AppData\Local\Temp\ipykernel_20628\4188038665.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False))


In [6]:
df_pytorch_classes = get_library_classes("torch", "../data/library_data/torch_data.json" , "../data/statistics/*")
df_pytorch_classes = df_pytorch_classes[:10]
df_to_latex(df=df_pytorch_classes)

\begin{tabular}{lrlrrl}
\toprule
      Class &  Count & Category &  \#HP &  AvgOptionsUsed & Most Used HP \\
\midrule
     Conv2d &  15072 &       nn &   11 &            4.95 &  in\_channels \\
     Linear &  14360 &       nn &    5 &            2.16 &  in\_features \\
 Sequential &  11247 &       nn &    1 &            0.93 &        *args \\
       ReLU &   9097 &       nn &    1 &            0.61 &      inplace \\
BatchNorm2d &   6507 &       nn &    7 &            1.34 & num\_features \\
  Parameter &   4812 &       nn &    2 &            1.17 &         data \\
 DataLoader &   4511 &    utils &   15 &            4.09 &      dataset \\
 ModuleList &   4169 &       nn &    1 &            0.50 &      default \\
    Dropout &   3694 &       nn &    2 &            0.95 &            p \\
       Adam &   2234 &    optim &    7 &            1.57 &      default \\
\bottomrule
\end{tabular}



C:\Users\Basti\AppData\Local\Temp\ipykernel_20628\4188038665.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False))


In [7]:
#df_all_classes = pd.concat([df_sklearn_classes, df_tf_classes, df_pytorch_classes])
#df_to_latex(df=df_sklearn_classes)

In [8]:
def get_library_methods(library_name: str, library_dir: str, project_dir: str) -> pd.DataFrame:
    with open(library_dir, "r", encoding="utf-8") as library_file:
        library_data = json.load(library_file)

    methods = []

    for project in glob.glob(project_dir):
        with open(project, "r", encoding="utf-8") as project_file:
            project_data = json.load(project_file)

            for file in project_data.keys():
                file_data = project_data[file]
                for library in file_data.keys():
                    if library == library_name:
                        module_data = file_data[library]
                        for key, _ in module_data.items():
                            if key[0].islower():
                                method_name_parts = key.split("_")[:-1]
                                method_name = "_".join(method_name_parts)
                                #for item in library_data:
                                #    if item["name"] == method_name:
                                methods.append(method_name)

    method_data = Counter(methods)
    df_methods = pd.DataFrame.from_dict(method_data, orient="index").reset_index()
    df_methods = df_methods.rename(columns={'index':'Method', 0:'Count'})
    df_methods = df_methods.sort_values(by=['Count'], ascending=False)

    # Compute number of args that can be set regarding the API data
    method_options = []
    methods = df_methods["Method"].to_list()
    categories = []

    for method in methods:
        try:
            method_data = next(filter(lambda x: x["name"] == method, library_data))
            method_options.append(len(method_data["params"]))
            parts = method_data["full_name"].split(".")
            if len(parts) <= 2:
                category = parts[0]
            else:
                category = parts[1]
            categories.append(category)
        except StopIteration:
            print("Could not find: ", method, library_name)
            method_options.append(0)
            categories.append("Unknown")
            continue

    df_methods["Category"] = categories
    df_methods["#Args"] = method_options

    # Compute average number of args used per method
    avg_method_args = []
    most_used_method_args = []

    for method in methods:
        avg_method_args_used = []
        method_args = []
        for project in glob.glob(project_dir):
            with open(project, "r", encoding="utf-8") as project_file:
                project_data = json.load(project_file)

                for file in project_data.keys():
                    file_data = project_data[file]
                    for library in file_data.keys():
                        if library == library_name:
                            module_data = file_data[library]
                            for module_name, data in module_data.items():
                                if module_name[0].islower():
                                    method_name_parts = module_name.split("_")[:-1]
                                    method_name = "_".join(method_name_parts)
                                    if method == method_name:
                                        counter = 0
                                        for arg in data.keys():
                                            if arg == "variable":
                                                continue

                                            if arg not in ("variable", "params", "class"):
                                                counter += 1

                                            if arg == "params":
                                                method_args.append("default")
                                            else:
                                                method_args.append(arg)
                                        
                                        avg_method_args_used.append(counter)
                                
        avg_method_args.append(round((sum(avg_method_args_used) / len(avg_method_args_used)),2))
        method_arg_data = Counter(method_args)
        try:
            most_common_num = method_arg_data.most_common(1)[0][1]
            args = [x[0] for x in method_arg_data.most_common() if x[1] == most_common_num]
            arg_str = ", ".join(args)
            most_used_method_args.append(arg_str)
        except IndexError:
            most_used_method_args.append("None")

    df_methods["AvgArgsUsed"] = avg_method_args
    df_methods["Most Used Args"] = most_used_method_args

    return df_methods

#df_sklearn_methods = get_library_methods("sklearn", "modules/sklearn_default_values.json", "data/statistics/*")
#df_sklearn_methods = df_sklearn_methods[:10]
#df_tf_methods = get_library_methods("tensorflow", "modules/tensorflow_default_values.json", "data/statistics/*")
#df_tf_methods = df_tf_methods[:10]
#df_torch_methods = get_library_methods("torch", "modules/torch_default_values.json", "data/statistics/*")
#df_torch_methods = df_torch_methods[:10]
#df_all_methods = pd.concat([df_sklearn_methods, df_tf_methods, df_torch_methods])
#df_to_latex(df=df_all_methods)


In [9]:
def get_algorithms(library_name: str, library_dir: str, project_dir: str) -> pd.DataFrame:
    with open(library_dir, "r", encoding="utf-8") as library_file:
        library_data = json.load(library_file)

    # Get Most used Class
    classes = []

    for project in glob.glob(project_dir):
        with open(project, "r", encoding="utf-8") as project_file:
            project_data = json.load(project_file)

            for file in project_data.keys():
                file_data = project_data[file]
                for library in file_data.keys():
                    if library == library_name:
                        module_data = file_data[library]
                        for key, value in module_data.items():
                            if key[0].isupper():
                                class_name = key.split("_")[0]
                                for item in library_data:
                                    if item["name"] == class_name:
                                        classes.append(class_name)


    class_data = Counter(classes)
    df_classes = pd.DataFrame.from_dict(class_data, orient="index").reset_index()
    df_classes = df_classes.rename(columns={'index':'Class', 0:'Count'})
    df_classes = df_classes.sort_values(by=['Count'], ascending=False)

    # Get Number of API Options and category
    categories = []
    class_options = []
    classes = df_classes["Class"].to_list()

    for ml_class in classes:
        try:
            class_data = next(filter(lambda x: x["name"] == ml_class, library_data))
            class_options.append(len(class_data["params"]))
            category = class_data["full_name"].split(".")[1]
            categories.append(category)
        except StopIteration:
            print("Could not find: ", ml_class)
            continue
            #raise StopIteration()

    df_classes["Category"] = categories
    df_classes["#HP"] = class_options

    # Compute average number of options used per class and most used option
    avg_class_options = []
    most_used_class_option = []
    # compute how often a algorithm is initialized with and without params
    init_with_params = []
    init_without_params = []

    for ml_class in classes:
        with_params = 0
        without_params = 0
        avg_class_options_used = []
        class_options = []
        for project in glob.glob(project_dir):
            with open(project, "r", encoding="utf-8") as project_file:
                project_data = json.load(project_file)

                for file in project_data.keys():
                    file_data = project_data[file]
                    for library in file_data.keys():
                        if library == library_name:
                            module_data = file_data[library]
                            for module_name, data in module_data.items():
                                if module_name[0].isupper():
                                    name = module_name.split("_")[0]
                                    if ml_class == name:
                                        counter = 0

                                        if "params" in data:
                                            without_params += 1
                                        else:
                                            with_params += 1

                                        for param in data.keys():
                                            if param == "variable":
                                                continue

                                            if param not in ("variable", "params", "class"):
                                                counter += 1

                                            if param == "params":
                                                class_options.append("default")
                                            else:
                                                class_options.append(param)
            
                                        avg_class_options_used.append(counter)


        init_with_params.append(with_params)
        init_without_params.append(without_params)                  
        avg_class_options.append(round((sum(avg_class_options_used) / len(avg_class_options_used)),2))
        class_option_data = Counter(class_options)
        try:
            #most_common_number = class_option_data.most_common(1)[0][1]
            #options = [elem[0] for elem in class_option_data.most_common() if elem[1] == most_common_number]
            #option_str = ", ".join(options)
            most_used_class_option.append(class_option_data.most_common(1)[0][0])
        except IndexError:
            most_used_class_option.append("None")

    df_classes["Init With Params"] = init_with_params
    df_classes["Init Without Params"] = init_without_params
    df_classes["AvgOptionsUsed"] = avg_class_options
    df_classes["Most Used HP"] = most_used_class_option

    return df_classes

#df_sklearn_ml_algo = get_algorithms("sklearn", "modules/sklearn_estimators.json", "data/statistics/*")
#df_sklearn_ml_algo = df_sklearn_ml_algo[:30]
#df_sklearn_ml_algo = df_sklearn_ml_algo[["Class", "Count", "Init With Params", "Init Without Params", "#HP", "AvgOptionsUsed"]]
#df_sklearn_ml_algo = df_sklearn_ml_algo.sort_values(by=['Count'], ascending=False)
#df_to_latex(df=df_sklearn_ml_algo)

In [10]:
def get_value_types(df, library_name, project_dir) -> pd.DataFrame:
    value_type = []

    for _, row in df_sklearn_ml_algo.iterrows():
        class_value_types = []

        df_class_name = row["Class"]
        df_class_option_name = row["Most Used HP"].split(",")[0]

        for project in glob.glob(project_dir):
                with open(project, "r", encoding="utf-8") as project_file:
                    project_data = json.load(project_file)

                    for file in project_data.keys():
                        file_data = project_data[file]
                        for library in file_data.keys():
                            module_data = file_data[library]
                            for module_name, data in module_data.items():
                                if library == library_name:
                                    module_data = file_data[library]
                                    for module_name, data in module_data.items():
                                        if module_name[0].isupper():
                                            name = module_name.split("_")[0]
                                            if df_class_name == name:
                                                for key, value in data.items():
                                                    if key == df_class_option_name:
                                                        #print(df_class_name, df_class_option_name, key, value)
                                                        possible_values = value["possible_values"]
                                                        if possible_values:
                                                            for x in possible_values:
                                                                class_value_types.append(x[1])
                                                        else:
                                                            class_value_types.append(value["type"])

        value_type_data = Counter(class_value_types)    
        top_types = value_type_data.most_common(1)
        try:
            if top_types:
                types = top_types[0][0]
                #types = [str(x[0][0]) for x in top_types]
                #types = ", ".join(types)
            else:
                types = "-"
        except Exception:
            print(row["Class"], row["Most Used HP"], value_type_data.most_common(3))
            print(top_types)
            types = "None"

        value_type.append(types)                                                   
    
    df["Common Value Types"] = value_type

    return df

#df = get_value_types(df_sklearn_ml_algo, "sklearn", "data/statistics/*")
#df_to_latex(df=df)

In [11]:
#df_sklearn_exp_settings = get_algorithms("sklearn", "modules/sklearn_exp_settings.json", "data/statistics/*")
#df_sklearn_ml_algo = df_sklearn_ml_algo[:30]

#df_sklearn_exp_settings = df_sklearn_exp_settings[["Class", "Category", "Count", "#HP", "AvgOptionsUsed", "Most Used HP"]]
#df_sklearn_exp_settings = df_sklearn_exp_settings.sort_values(by=['Count'], ascending=False)

#df_sklearn_exp_settings = get_value_types(df_sklearn_exp_settings, "sklearn", "statistics/*")

#df_to_latex(df=df_sklearn_exp_settings)

In [12]:
def count_init_type_of_algorithms(library_dir: str, project_dir: str) -> pd.DataFrame:
    with open(library_dir, "r", encoding="utf-8") as library_file:
        library_data = json.load(library_file)

    # Get Most used Class
    classes = []

    for project in glob.glob(project_dir):
        with open(project, "r", encoding="utf-8") as project_file:
            project_data = json.load(project_file)

            for file in project_data.keys():
                file_data = project_data[file]
                for library in file_data.keys():
                    if library == "sklearn":
                        module_data = file_data[library]
                        for key, value in module_data.items():
                            if key[0].isupper():
                                class_name = key.split("_")[0]
                                for item in library_data:
                                    if item["name"] == class_name:
                                        classes.append(class_name)
                                        


    class_data = Counter(classes)
    df_classes = pd.DataFrame.from_dict(class_data, orient="index").reset_index()
    df_classes = df_classes.rename(columns={'index':'Class', 0:'Count'})
    df_classes = df_classes.sort_values(by=['Count'], ascending=False)
    

    init_with_params = []
    init_without_params = []
    classes = df_classes["Class"].to_list()

    for ml_class in classes:
        with_params = 0
        without_params = 0
        for project in glob.glob(project_dir):
            with open(project, "r", encoding="utf-8") as project_file:
                project_data = json.load(project_file)

                for file in project_data.keys():
                    file_data = project_data[file]
                    for library in file_data.keys():
                        if library == "sklearn":
                            module_data = file_data[library]
                            for module_name, data in module_data.items():
                                if module_name[0].isupper():
                                    name = module_name.split("_")[0]
                                    if ml_class == name:
                                        if "params" in data:
                                            without_params += 1
                                        else:
                                            with_params += 1
        

        init_with_params.append(with_params)
        init_without_params.append(without_params)

    
    df_classes["Init With Params"] = init_with_params
    df_classes["Init Without Params"] = init_without_params

    return df_classes

#df_sklearn_ml_algo = count_init_type_of_algorithms("modules/sklearn_estimators.json", "data/statistics/*")
#df_sklearn_ml_algo = df_sklearn_ml_algo[:30]
#df_sklearn_ml_algo = df_sklearn_ml_algo.sort_values(by=['Count'], ascending=False)

#df_to_latex(df=df_sklearn_ml_algo)